In [1]:
from choice_model import ChoiceModel

folder = 'orig_graph'

def predict(period):
    model = ChoiceModel()
    prev_period_file = f'{folder}/{folder}_{period-1}.pkl'
    new_model = ChoiceModel(graph_path=prev_period_file) 
    graph = model.get_period(period)
    new_nodes = [n for n in graph.nodes if n not in list(new_model.graph.nodes)]
    new_nodes = [n for n in new_nodes if graph.nodes[n]['type'] != 'Organization']
    test_graph = new_model.graph.copy()
    test_graph = model.predict_links(test_graph, new_nodes, file_name=f"{folder}/{folder}_{period}.pkl")
    new_model = ChoiceModel(graph_path=f"{folder}/{folder}_{period}.pkl")
    issolated_nodes = new_model.get_isolated_nodes()
    if len(issolated_nodes) > 0:
        test_graph = new_model.graph.copy()
        test_graph = model.predict_links(test_graph, issolated_nodes,file_name=f"{folder}/{folder}_{period}.pkl",k1=10,k2=5,top_k=5)
        new_model.remove_isolated_nodes()
    new_model.visualize(file_name=f"{folder}/{folder}_{period}.html")
    PPI, CVI, DCI = new_model.evaluate(test_graph)
    print(f"Period {period} PPI: {PPI} CVI: {CVI} DCI: {DCI}")


In [2]:
predict(1)

 30%|██▉       | 19/64 [03:48<08:59, 11.98s/it]

'answer'


100%|██████████| 1/1 [00:08<00:00,  8.95s/it]

'answer'
Period 1 PPI: 0.2549393406463926 CVI: 3.52 DCI: 0.07317073170731707


In [ ]:
# for i in range(1,5):
#     predict(i)

  8%|▊         | 6/76 [00:53<10:40,  9.16s/it]

'answer'


 17%|█▋        | 13/76 [02:12<09:26,  8.99s/it]

'answer'


 42%|████▏     | 32/76 [06:25<08:39, 11.80s/it]

...

'answer'


 35%|███▌      | 59/167 [11:06<35:42, 19.84s/it]

'answer'


 39%|███▉      | 65/167 [12:58<20:20, 11.97s/it]


KeyboardInterrupt: 